In [1]:
import glob

import numpy as np
import pandas as pd

from grafting_classifier import GraftingClassifier
from sklearn.linear_model import SGDClassifier
from ogfs_classifier import OGFSClassifier
from osfs_classifier import OSFSClassifier
#from dpp_classifier import DPPClassifier
#from dpp_classifier_dppsample import DPPClassifier as DPPClassifier0
#from dpp_classifier_mitra import DPPClassifier as DPPClassifier2
#from dpp_classifier_ogfs_dppsample import DPPClassifier as DPPClassifier3
from dpp_classifier_supervised import DPPClassifier as DPPClassifier3

from sklearn.metrics import log_loss, accuracy_score

#import dask.dataframe as dd
#import dask.array as da

In [2]:
class_train = glob.glob("uci/*_train.csv")
print(class_train)

['uci\\Ionosphere_train.csv', 'uci\\spambase_train.csv', 'uci\\spectf_train.csv', 'uci\\wdbc_train.csv']


In [3]:
def train_label(fname):
    targetname = fname.replace(".csv", ".labels")
    return pd.read_csv(targetname)

In [4]:
def get_performance(mod, fpath, mod_name):
    train1 = pd.read_csv(fpath).fillna(0)
    y = np.array(train_label(fpath)).flatten()
    
    # simulate streaming...
    # try splitting into groups of ~10,
    # if there is no splits, try ~5.
    train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/10.0) + 1)
    if len(train1_cols) == 1:
        train1_cols = np.array_split(range(train1.shape[1]), int(train1.shape[1]/5.0) + 1)
    all_cols = []

    #mod = GraftingClassifier(max_iter=5)
    if mod_name == 'Base':
        mod.fit(train1, y)
        results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
        return results
    
    # lets normalise the dataset...
    train1 = (train1 - train1.mean())/(np.maximum(train1.std(), 1))
    for idx, collist in enumerate(train1_cols):
        if idx == 0:
            column_list = list(np.array(list(train1.columns))[collist])
            mod.fit(train1[column_list], y)
            all_cols.extend(list(collist))
        else:
            all_cols.extend(list(collist))
            column_list = list(np.array(list(train1.columns))[all_cols])
            mod.partial_fit(train1[column_list], y)
        
        # debugging
        print_cond = True if idx % int((len(train1_cols)/10)+1) == 0 else False
        if mod_name in ['Fast_OSFS', 'DPP', 'DPP3', 'OGFS'] and print_cond:
            print("\tmodel: {}, iter: {}".format(mod_name, idx))
        
        # for fast osfs
    if mod_name == 'Fast_OSFS':
        mod._redundancy(train1, y, mode='all')
    
    results = {'accuracy': accuracy_score(y, mod.predict(train1)), 
               'logloss': log_loss(y, mod.predict_proba(train1)), 
               'feat_dim': mod.coef_.flatten().shape}
    return results

In [5]:
def create_models():
    return [
    #('Grafting', GraftingClassifier(max_iter=5, random_state=42)), 
    #('DPP', DPPClassifier(max_iter=5, random_state=42)), 
    #('DPP0', DPPClassifier0(max_iter=5, random_state=42)), 
    #('DPP2', DPPClassifier2(max_iter=5, random_state=42)),
    ('DPP3', DPPClassifier3(max_iter=5, random_state=42)),
    #('OGFS', OGFSClassifier(max_iter=5, random_state=42)),
    #('OSFS', OSFSClassifier(max_iter=5, random_state=42, fast_osfs=False)),
    #('Fast_OSFS', OSFSClassifier(max_iter=5, random_state=42)),
    ('Base', SGDClassifier(loss='log', max_iter=5, random_state=42))
]

In [10]:
ex_dat = class_train[0]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    print(nm, get_performance(mod, ex_dat, mod_name=nm))

uci\Ionosphere_train.csv (351, 34)
	model: DPP3, iter: 0
(351, 18)
	model: DPP3, iter: 1
(351, 26)
	model: DPP3, iter: 2
(351, 34)
	model: DPP3, iter: 3
DPP3 {'accuracy': 0.87749287749287752, 'logloss': 1.0831890572014167, 'feat_dim': (34,)}
Base {'accuracy': 0.91737891737891741, 'logloss': 1.9154349277794036, 'feat_dim': (34,)}


In [7]:
ex_dat = class_train[1]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    print(nm, get_performance(mod, ex_dat, mod_name=nm))


uci\spambase_train.csv (4601, 57)
	model: DPP3, iter: 0
(4601, 20)
	model: DPP3, iter: 1
(4601, 30)
	model: DPP3, iter: 2
(4601, 28)
	model: DPP3, iter: 3
(4601, 30)


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2397: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\morestats.py:2422: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


	model: DPP3, iter: 4
(4601, 45)
	model: DPP3, iter: 5
DPP3 {'accuracy': 0.90871549663116713, 'logloss': 0.30587131671855494, 'feat_dim': (44,)}
Base {'accuracy': 0.708324277331015, 'logloss': 10.070713136799997, 'feat_dim': (57,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [8]:
ex_dat = class_train[2]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    print(nm, get_performance(mod, ex_dat, mod_name=nm))

uci\spectf_train.csv (267, 44)
	model: DPP3, iter: 0
(267, 18)


C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming_take2\SPEC.py:114: RuntimeWarning: invalid value encountered in power
  d1 = np.power(np.array(W.sum(axis=1)), -0.5)
C:\Users\chapm\Documents\GitHub\sklearn-recipes\streaming_take2\SPEC.py:116: RuntimeWarning: invalid value encountered in power
  d2 = np.power(np.array(W.sum(axis=1)), 0.5)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


	model: DPP3, iter: 1
(267, 27)
	model: DPP3, iter: 2
(267, 28)
	model: DPP3, iter: 3
(267, 26)
	model: DPP3, iter: 4
DPP3 {'accuracy': 0.79400749063670417, 'logloss': 0.80127978578316683, 'feat_dim': (11,)}
Base {'accuracy': 0.79400749063670417, 'logloss': 7.1147292199254215, 'feat_dim': (44,)}


In [9]:
ex_dat = class_train[3]
print(ex_dat, pd.read_csv(ex_dat).shape)
models = create_models()
for nm, mod in models:
    print(nm, get_performance(mod, ex_dat, mod_name=nm))

uci\wdbc_train.csv (569, 30)
	model: DPP3, iter: 0
(569, 16)
	model: DPP3, iter: 1
(569, 22)
	model: DPP3, iter: 2
(569, 23)
	model: DPP3, iter: 3
DPP3 {'accuracy': 0.91036906854130051, 'logloss': 0.24975493601716769, 'feat_dim': (9,)}
Base {'accuracy': 0.91564147627416526, 'logloss': 2.9136401879713767, 'feat_dim': (30,)}


C:\Users\chapm\Anaconda3\envs\skrecipe\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
